<a href="https://colab.research.google.com/github/micaelCZ/Paper_Repositorio/blob/main/A%20CNN%2BENC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
datapath = 'https://raw.githubusercontent.com/micaelCZ/Paper_Repositorio/main/dataset_nuevos/Escenario1.csv'
dataframe = pd.read_csv(datapath,low_memory=False, sep=',')
#Visualización de los 5 primeros datos
dataframe.head()


,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,...,Bwd IAT Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,10.0.2.15,53913,216.58.208.46,80,6,435,0.0,4597.701149,435.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
1,10.0.2.15,53913,216.58.208.46,80,6,259,0.0,7722.007722,259.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
2,10.0.2.15,53913,216.58.208.46,80,6,891,0.0,2244.668911,891.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
3,10.0.2.15,53913,216.58.208.46,80,6,1074,0.0,1862.197393,1074.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
4,10.0.2.15,53913,216.58.208.46,80,6,315,0.0,6349.206349,315.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR


In [2]:
# Normalización de los datos
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:,feature_name]= pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()   
        if (max_value - min_value) > 0:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:,feature_name] = (df.loc[:,feature_name]- min_value)    
    return df


In [3]:
#Muestra el tamaño del dataframe (número de filas y columnas)
print (dataframe.shape)
dataframe = dataframe.reindex(np.random.permutation(dataframe.index)).copy()
print(dataframe.describe())
print (list(dataframe))

(67834, 29)
        Source Port   Destination Port      Protocol   Flow Duration  \
count  67834.000000       67834.000000  67834.000000    6.783400e+04   
mean   37912.753324       11566.395967     12.167291    2.991884e+06   
std    20986.077326       18374.765123      5.459410    4.063005e+06   
min       21.000000          21.000000      6.000000    0.000000e+00   
25%    19305.000000         137.000000      6.000000    4.435975e+04   
50%    43677.000000         443.000000     17.000000    4.108570e+05   
75%    54685.000000       16311.000000     17.000000    7.325550e+06   
max    65534.000000       65514.000000     17.000000    1.000000e+07   

        Flow Bytes/s   Flow Packets/s   Flow IAT Mean   Flow IAT Std  \
count   6.783200e+04     6.783400e+04    6.783400e+04   6.783400e+04   
mean             inf              inf    3.155927e+05   2.209662e+05   
std              NaN              NaN    6.988069e+05   6.409506e+05   
min     0.000000e+00     2.002581e-01    0.000000e+

In [4]:
#Obtiene las claves del dataframe (los nombres de las columnas)
keys = dataframe.keys()
data_to_process = dataframe[keys[4:len(keys) - 1]].copy()
x_normalised = dfNormalize(data_to_process)
print(x_normalised.describe())

<ipython-input-2-99d760175aeb>:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)


           Protocol   Flow Duration   Flow Bytes/s   Flow Packets/s  \
count  67834.000000    67834.000000        67830.0          67828.0   
mean       0.560663        0.299188            0.0              0.0   
std        0.496310        0.406300            0.0              0.0   
min        0.000000        0.000000            0.0              0.0   
25%        0.000000        0.004436            0.0              0.0   
50%        1.000000        0.041086            0.0              0.0   
75%        1.000000        0.732555            0.0              0.0   
max        1.000000        1.000000            0.0              0.0   

        Flow IAT Mean   Flow IAT Std   Flow IAT Max   Flow IAT Min  \
count    67834.000000   67834.000000   67834.000000   67834.000000   
mean         0.031600       0.031363       0.089855       0.019491   
std          0.069971       0.090973       0.173880       0.057865   
min          0.000000       0.000000       0.000000       0.000000   
25%       

In [5]:
#Cambiar las etiquetas de los datos de ‘nonTOR’ y ‘TOR’ a 1 y 0 respectivamente.
change_labels = lambda x: 1 if x == 'nonTOR' else 0


In [6]:
#crea una nueva variable llamada y_normalised que contiene los valores de la columna 
#‘label’ del dataframe original después de aplicar la función change_labels.
y_normalised = dataframe['label'].apply(change_labels)


In [7]:
# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(x_normalised, y_normalised, test_size=0.3, random_state=42)


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.layers import Reshape, UpSampling1D
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.callbacks import EarlyStopping

# Encoder
def create_model(dropout_rate=0.1):
    model = Sequential([
        Conv1D(64, 3, input_shape=(X_train.shape[1], 1), padding="same", activation="relu", kernel_constraint=MaxNorm(3)),
        MaxPooling1D(2),
        Dropout(dropout_rate),
        Conv1D(32, 3, padding="same", activation="relu", kernel_constraint=MaxNorm(3)),
        MaxPooling1D(2),
        Dropout(dropout_rate),
        Conv1D(16, 3, padding="same", activation="relu", kernel_constraint=MaxNorm(3)),
        MaxPooling1D(2),
        Dropout(dropout_rate),
        Flatten(),
        Dense(1, activation="sigmoid"),
    ])

    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model



In [9]:
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Definir los hiperparámetros y sus valores a probar
param_grid = {
    'batch_size': [16, 32],
    'epochs': [10, 20],
    'dropout_rate': [0.1, 0.2, 0.3],
}

# Crear el modelo de Keras con la función create_model y pasarla como estimador en GridSearchCV
model = KerasClassifier(build_fn=create_model, dropout_rate=0.1) # Establecer el valor predeterminado de dropout_rate
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)

<ipython-input-10-127bce1b72f4>:12: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, dropout_rate=0.1) # Establecer el valor predeterminado de dropout_rate


Epoch 1/10
1979/1979 [==============================] - 15s 7ms/step - loss: nan - accuracy: 0.1208
Epoch 2/10
1979/1979 [==============================] - 13s 6ms/step - loss: nan - accuracy: 0.1181
Epoch 3/10
1979/1979 [==============================] - 7s 4ms/step - loss: nan - accuracy: 0.1181
Epoch 4/10
1979/1979 [==============================] - 8s 4ms/step - loss: nan - accuracy: 0.1181
Epoch 5/10
1979/1979 [==============================] - 7s 3ms/step - loss: nan - accuracy: 0.1181
Epoch 6/10
1979/1979 [==============================] - 8s 4ms/step - loss: nan - accuracy: 0.1181
Epoch 7/10
1979/1979 [==============================] - 7s 3ms/step - loss: nan - accuracy: 0.1181
Epoch 8/10
1979/1979 [==============================] - 8s 4ms/step - loss: nan - accuracy: 0.1181
Epoch 9/10
1979/1979 [==============================] - 8s 4ms/step - loss: nan - accuracy: 0.1181
Epoch 10/10
990/990 [==============================] - 2s 2ms/step - loss: nan - accuracy: 0.1180
Epoch 1/1

In [11]:
# Obtener los mejores hiperparámetros y su rendimiento en los datos de entrenamiento
best_params = grid_result.best_params_
best_score = grid_result.best_score_

# Imprimir los mejores hiperparámetros y su rendimiento
print("Mejores hiperparámetros encontrados: ", best_params)
print("Rendimiento del modelo con los mejores hiperparámetros: ", best_score)


Mejores hiperparámetros encontrados:  {'batch_size': 16, 'dropout_rate': 0.1, 'epochs': 10}
Rendimiento del modelo con los mejores hiperparámetros:  0.11804230014483134


In [12]:
batch_size = 16
steps_per_epoch = (X_train.shape[0]//batch_size)
validation_steps = (x_normalised.shape[0] // batch_size)

es = EarlyStopping(monitor='accuracy', mode='min', verbose=1, patience=200)


In [13]:
# Obtener el mejor estimador del resultado de la optimización de hiperparámetros
best_model = grid_result.best_estimator_

# Entrenar el modelo con los mejores hiperparámetros
best_model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'])


Epoch 1/10
2968/2968 [==============================] - 24s 8ms/step - loss: nan - accuracy: 0.1639
Epoch 2/10
2968/2968 [==============================] - 16s 5ms/step - loss: nan - accuracy: 0.1180
Epoch 3/10
2968/2968 [==============================] - 11s 4ms/step - loss: nan - accuracy: 0.1180
Epoch 4/10
2968/2968 [==============================] - 12s 4ms/step - loss: nan - accuracy: 0.1180
Epoch 5/10
2968/2968 [==============================] - 12s 4ms/step - loss: nan - accuracy: 0.1180
Epoch 6/10
2968/2968 [==============================] - 12s 4ms/step - loss: nan - accuracy: 0.1180
Epoch 7/10
2968/2968 [==============================] - 12s 4ms/step - loss: nan - accuracy: 0.1180
Epoch 8/10
2968/2968 [==============================] - 12s 4ms/step - loss: nan - accuracy: 0.1180
Epoch 9/10
2968/2968 [==============================] - 12s 4ms/step - loss: nan - accuracy: 0.1180
Epoch 10/10
2968/2968 [==============================] - 12s 4ms/step - loss: nan - accuracy: 0.1180

In [14]:
%%time 

# Entrenar el modelo
model.fit(X_train, y_train,
    steps_per_epoch=steps_per_epoch,
    epochs = 35,
    validation_data=(X_test, y_test),
    validation_steps=validation_steps,
    batch_size = batch_size,
    callbacks=[es],
    verbose=1)

Epoch 1/35
2955/2967 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.2208

2967/2967 [==============================] - 15s 5ms/step - loss: nan - accuracy: 0.2206 - val_loss: nan - val_accuracy: 0.1198
Epoch 2/35
2967/2967 [==============================] - 12s 4ms/step - loss: nan - accuracy: 0.1181
Epoch 3/35
2967/2967 [==============================] - 12s 4ms/step - loss: nan - accuracy: 0.1181
Epoch 4/35
2967/2967 [==============================] - 12s 4ms/step - loss: nan - accuracy: 0.1180
Epoch 5/35
2967/2967 [==============================] - 12s 4ms/step - loss: nan - accuracy: 0.1180
Epoch 6/35
2967/2967 [==============================] - 12s 4ms/step - loss: nan - accuracy: 0.1182
Epoch 7/35
2967/2967 [==============================] - 11s 4ms/step - loss: nan - accuracy: 0.1181
Epoch 8/35
2967/2967 [==============================] - 12s 4ms/step - loss: nan - accuracy: 0.1180
Epoch 9/35
2967/2967 [==============================] - 12s 4ms/step - loss: nan - accuracy: 0.1180
Epoch 10/35
2967/2967 [==============================] - 11s 4ms/step - 

In [15]:
# Obtener el mejor estimador del resultado de la optimización de hiperparámetros
best_model = grid_result.best_estimator_

# Evaluar el modelo en datos de prueba
y_pred = best_model.predict(X_test)
accuracy = best_model.score(X_test, y_test)
print('test_accuracy: %.2f' % (accuracy * 100.0))


1272/1272 [==============================] - 2s 2ms/step - loss: nan - accuracy: 0.1198
test_accuracy: 11.98


In [16]:
from sklearn.metrics import classification_report

# Hacer predicciones sobre los datos de prueba
y_pred = model.predict(X_test)

# Convertir las probabilidades en etiquetas binarias (0 o 1) usando un umbral de 0.5
y_pred_binary = (y_pred > 0.5).astype(int)

# calcular métricas de evaluación
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score
acc = accuracy_score(y_test, y_pred_binary)*100
f1 = f1_score(y_test, y_pred_binary, average='weighted')*100
prec = precision_score(y_test, y_pred_binary, average='weighted')*100
rec = recall_score(y_test, y_pred_binary, average='weighted')*100

# imprimir las métricas
print("Accuracy:", acc)
print("F1 score:", f1)
print("Precision:", prec)
print("Recall:", rec)

636/636 [==============================] - 1s 2ms/step
Accuracy: 11.984669058031546
F1 score: 2.5652135000034173
Precision: 1.4363229243053877
Recall: 11.984669058031546


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
